In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import normaltest, chi2, ks_1samp, norm
from ATARI.ModelData.experimental_model import Experimental_Model
# from ATARI.sammy_interface import sammy_classes, sammy_functions
from ATARI.ModelData.particle_pair import Particle_Pair
from ATARI.ModelData.measurement_models.transmission_rpi import Transmission_RPI
from ATARI.ModelData.measurement_models.capture_yield_rpi import Capture_Yield_RPI

from ATARI.syndat.syndat_model import Syndat_Model
from ATARI.syndat.data_classes import syndatOPT
from ATARI.syndat import tests # noise_distribution_test2, noise_distribution_test, no_sampling_returns_true_test
from ATARI.syndat.general_functions import approximate_neutron_spectrum_Li6det

import importlib


In [2]:
%matplotlib widget

In [83]:
pair = Particle_Pair()
pert = 1500
print_out = True

da_avg = 3.0e02 * 1e-2; db_avg = 6.005157275119e-02 * 1e-2


model_par = {
    'trigs' :   (1e8,  0),
    'trigo' :   (1e8,  0),
    'm1'    :   (1,     0.0001) ,#0.016
    'm2'    :   (1,     0.0001) ,#0.008
    'm3'    :   (1,     0.0001) ,#0.018
    'm4'    :   (1,     0.0001) ,#0.005
    'ks'    :   (0.563, 0.0002), #0.02402339737495515
    'ko'    :   (1.471, 0.0005), #0.05576763648617445
    'b0s'   :   (9.9,   0.008) ,#0.1
    'b0o'   :   (13.4,  0.01) ,#0.7
    # 'ks'    : (1.653,          0.0002413126885507)
    # 'ko'    : (1.882,          0.0004334047072747)
   'a_b'    : ([582.7768594580712, 0.05149689096209191],
                                # [[5.14395753e+02,  1.42659922e-1],
                                # [1.42659922e-1,   2.19135003e-06]]
                        [[da_avg**2, 9.930753594644e-01*da_avg*db_avg], [9.930753594644e-01*da_avg*db_avg, db_avg**2]])
    }



In [ ]:
importlib.reload(tests)
# def test_random_energy(self):

energy_grid = np.sort(np.random.default_rng().uniform(10,3000,10)) #np.linspace(min(energy_range),max(energy_range),10) # energy below 10 has very low counts due to approximate open spectrum
df_true = pd.DataFrame({'E':energy_grid, 'true':np.random.default_rng().uniform(0.01,1.0,10)})
df_true.sort_values('E', ascending=True, inplace=True)

generative_model = Transmission_RPI(**model_par)
reductive_model = Transmission_RPI(**model_par)

synOPT = syndatOPT(sampleRES=False, calculate_covariance=True, explicit_covariance=True, sampleTMP=True, smoothTNCS=True) 
exp_model = Experimental_Model(energy_grid=energy_grid, energy_range = [min(energy_grid), max(energy_grid)])
for measurement_model in [generative_model, reductive_model]:
    measurement_model.approximate_unknown_data(exp_model=exp_model, smooth=True, check_trig=True, overwrite=True)
SynMod = Syndat_Model(generative_experimental_model=exp_model, generative_measurement_model=generative_model, reductive_measurement_model=reductive_model, options=synOPT)


fig, mean_of_residual, norm_test_on_residual, kstest_on_chi2 = tests.noise_distribution_test2(SynMod, df_true = df_true, ipert=pert, print_out=print_out, return_figs=True, title="Transmission (exponential)") #noise_distribution_test(SynMod, print_out=True)

axes = fig.get_axes()
for ax in axes:
    ax.set_facecolor('#0f0f0f15')  
    ax.grid(True, linestyle='-', color='white', alpha=1.0, zorder=-10)  # Add grid with style
    # ax.tick_params(axis='both', which='both', length=0, width=0)
fig.tight_layout()

In [ ]:
energy_grid = np.sort(np.random.default_rng().uniform(10,3000,10)) #np.linspace(min(energy_range),max(energy_range),10) # energy below 10 has very low counts due to approximate open spectrum
df_true = pd.DataFrame({'E':energy_grid, 'true':np.random.default_rng().uniform(0.01,1.0,10)})
df_true.sort_values('E', ascending=True, inplace=True)

generative_model = Transmission_RPI(**model_par, bkg_func = 'power')
reductive_model = Transmission_RPI(**model_par, bkg_func = 'power')

synOPT = syndatOPT(sampleRES=False, calculate_covariance=True, explicit_covariance=True, sampleTMP=True, smoothTNCS=True) 
exp_model = Experimental_Model(energy_grid=energy_grid, energy_range = [min(energy_grid), max(energy_grid)])
for measurement_model in [generative_model, reductive_model]:
    measurement_model.approximate_unknown_data(exp_model=exp_model, smooth=True, check_trig=True, overwrite=True)
SynMod = Syndat_Model(generative_experimental_model=exp_model, generative_measurement_model=generative_model, reductive_measurement_model=reductive_model, options=synOPT)


fig, mean_of_residual, norm_test_on_residual, kstest_on_chi2 = tests.noise_distribution_test2(SynMod, df_true = df_true, ipert=pert, print_out=print_out, return_figs=True, title="Transmission (power)") #noise_distribution_test(SynMod, print_out=True)

axes = fig.get_axes()
for ax in axes:
    ax.set_facecolor('#0f0f0f15')  
    ax.grid(True, linestyle='-', color='white', alpha=1.0, zorder=-10)  # Add grid with style
    # ax.tick_params(axis='both', which='both', length=0, width=0)
fig.tight_layout()

In [ ]:
pair = Particle_Pair()
pert = 1500
print_out = True

model_par = {
        'trigs' :   (1e10,  0),
        'trigo' :   (1e10,  0),
        }

exp_model = Experimental_Model(channel_widths={"maxE": [3000],"chw": [1200.0],"dchw": [0.8]})
df_true = pd.DataFrame({'E': exp_model.energy_grid, 'true': np.random.default_rng().uniform(0.01,1.0,len(exp_model.energy_grid)) })#np.ones(len(exp_model.energy_grid))*0.9 })

generative_model = Capture_Yield_RPI(**model_par)
reductive_model = Capture_Yield_RPI(**model_par)
for measurement_model in [generative_model, reductive_model]:
    measurement_model.approximate_unknown_data(exp_model=exp_model, smooth=True, check_trig=True, overwrite=True)

synOPT = syndatOPT(smoothTNCS=True, sampleRES=False, calculate_covariance=True, explicit_covariance=True, sampleTMP=True) 
SynMod = Syndat_Model(generative_experimental_model=exp_model, generative_measurement_model=generative_model, reductive_measurement_model=reductive_model, options=synOPT)
fig, mean_of_residual, norm_test_on_residual, kstest_on_chi2 = tests.noise_distribution_test2(SynMod, df_true = df_true, ipert=pert, print_out=print_out, return_figs=True, title="Capture Yield") 

axes = fig.get_axes()
for ax in axes:
    ax.set_facecolor('#0f0f0f15')  
    ax.grid(True, linestyle='-', color='white', alpha=1.0, zorder=-10)  # Add grid with style
    # ax.tick_params(axis='both', which='both', length=0, width=0)
fig.tight_layout()


In [ ]:

    # def test_with_given_TNCS(self):

    #     exp_model = Experimental_Model(channel_widths={"maxE": [3000],"chw": [1200.0],"dchw": [0.8]})
    #     df_true = pd.DataFrame({'E': exp_model.energy_grid, 'tof':exp_model.tof_grid,'true': np.random.default_rng().uniform(0.1,1.0,len(exp_model.energy_grid)) })#np.ones(len(exp_model.energy_grid))*0.9 })

    #     open_df = pd.DataFrame({"E":exp_model.energy_grid, 
    #                             'tof': exp_model.tof_grid,
    #                             'bw': abs(np.append(np.diff(exp_model.tof_grid), np.diff(exp_model.tof_grid)[-1])*1e-9),
    #                            'ct':np.ones(len(exp_model.energy_grid))*1e10,
    #                            'dct': np.ones(len(exp_model.energy_grid))*np.sqrt(1e10)})
    #     bkg_df = pd.DataFrame({"E":exp_model.energy_grid, 
    #                             'tof': exp_model.tof_grid,
    #                             'bw': abs(np.append(np.diff(exp_model.tof_grid), np.diff(exp_model.tof_grid)[-1])*1e-9),
    #                            'ct':np.ones(len(exp_model.energy_grid))*1e1,
    #                            'dct': np.ones(len(exp_model.energy_grid))*np.sqrt(1e1)})
    #     # bkg_df = approximate_neutron_spectrum_Li6det(exp_model.energy_grid, True, exp_model.FP[0], exp_model.t0[0], self.model_par['trigo'][0])

    #     generative_model = Capture_Yield_RPI(**self.model_par,
    #                                         background_spectrum_bg = bkg_df,
    #                                         incident_neutron_spectrum_f = open_df,
    #                                         background_spectrum_bf =bkg_df 
    #                                         )
    #     reductive_model = Capture_Yield_RPI(**self.model_par, 
    #                                         background_spectrum_bg = bkg_df,
    #                                         incident_neutron_spectrum_f = open_df,
    #                                         background_spectrum_bf = bkg_df
    #                                         )
    #     # for measurement_model in [generative_model, reductive_model]:
    #     #     measurement_model.approximate_unknown_data(exp_model=exp_model, smooth=True, check_trig=True, overwrite=True)

    #     synOPT = syndatOPT(sampleRES=False, calculate_covariance=True, explicit_covariance=True, sampleTMP=True, smoothTNCS=True) 
    #     SynMod = Syndat_Model(generative_experimental_model=exp_model, generative_measurement_model=generative_model, reductive_measurement_model=reductive_model, options=synOPT)
        
    #     mean_of_residual, norm_test_on_residual, kstest_on_chi2 = noise_distribution_test2(SynMod, df_true = df_true, ipert=self.pert, print_out=self.print_out) #noise_distribution_test(SynMod, print_out=True)
    #     # self.assertTrue( np.isclose(mean_of_residual, 0, atol=1e-1), 
    #     #                 "Mean of residuals is not 0")
    #     # self.assertTrue( norm_test_on_residual.pvalue>1e-5, 
    #     #                 f"Normalized residuals are not standard normal: pvalue = {norm_test_on_residual.pvalue}")
    #     # self.assertTrue( kstest_on_chi2.pvalue>1e-5, 
    #     #                 "Chi2 of data does not have appropriate DOF")


